In [3]:
import pandas as pd
import numpy as np 
from scipy.stats import norm

In [4]:
df = pd.read_csv('C:/Users/adity/OneDrive/Documents/Monotonic Binning/train.csv')

In [5]:
df.shape

(219129, 32)

In [17]:
df.columns

Index(['id', 'Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9',
       'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19',
       'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [18]:
# WOE & IV
def woe_iv(df,col, bins):
    bins_x = pd.qcut(df[col], bins, duplicates='drop', retbins=True)
    binned_df_x = pd.DataFrame(bins_x[0])
    binned_df_x['Class'] = df['Class']
    group_df_x = binned_df_x.groupby(col).aggregate(['sum','count']).reset_index()
    group_df_x.columns = ['bins', 'events', 'total']
    group_df_x['non events'] = group_df_x['total'] - group_df_x['events']
    good_sum = group_df_x['non events'].sum()
    bad_sum = group_df_x['events'].sum()
    group_df_x['% of events'] = group_df_x['events']/bad_sum
    group_df_x['% of non events'] = group_df_x['non events']/good_sum
    group_df_x['WoE'] = np.log(group_df_x['% of non events']/group_df_x['% of events'])
    group_df_x['IV'] = group_df_x['WoE'] * (group_df_x['% of non events']-group_df_x['% of events'])
    iv_val = group_df_x['IV'].sum()
    return group_df_x, iv_val

In [19]:
df_temp = df[['V19','Class']]
df_temp = df_temp.sort_values(by=['V19'])
df_temp['m'] = df_temp['Class']

feature = df_temp['V19'].to_list()
mean = df_temp['m'].to_list()
Class = df_temp['Class'].to_list()
p_val_thresh = 0.30 #how similar the bins can be
min_bin_size = 8
ind_var_rate = 0.4

In [20]:
stack = []
for i in range(0,len(feature)):
    if len(stack)==0 or stack[-1][-1] <= mean[i]:
        stack.append([i,i,mean[i]])
    elif stack[-1][-1] > mean[i]:
        temp = stack.pop()
        l, r = temp[0], temp[1]
        if l==r:
            new_m = ((temp[-1] + mean[i]) / 2) 
        else:   
            new_m = ((r-l+1)*(temp[-1]) + mean[i]) / (r-l+1 + 1)
        stack.append([l, i, new_m])
        merge, j = True, len(stack)-2
        while merge:
            if len(stack) == 1 or j == len(stack)-1:
                merge = False
            elif stack[j][-1] <= stack[j+1][-1]:
                j += 1 
            elif stack[j][-1] > stack[j+1][-1]:
                temp_2 = stack.pop()
                l_2, r_2 = temp_2[0],temp_2[1]
                temp_1 = stack.pop()
                l_1, r_1 = temp_1[0], temp_1[1]
                if l_1 == r_1 and l_2 == r_2:
                    new_m_1 = (temp_1[-1] + temp_2[-1])/2
                elif l_1 == r_1:
                    new_m_1 = (temp_1[-1] + (r_2-l_2+1)*temp_2[-1]) / (1 + r_2-l_2+1)
                elif l_2 == r_2:
                    new_m_1 = ((r_1-l_1)*temp_1[-1] + temp_2[-1]) / (1 + r_1-l_1)
                else: 
                    new_m_1 = ((r_1-l_1)*temp_1[-1] + (r_2-l_2)*temp_2[-1]) / (r_1-l_1 + r_2-l_2)
                stack.append([l_1,r_2,new_m_1])
                j -= 1 

In [21]:
def p_value_cal(bin1, bin2, Class):
    m_1,m_2 = bin1[2], bin2[2]
    bin_1_data = Class[bin1[0]:bin1[1]+1]
    bin_2_data = Class[bin2[0]:bin2[1]+1]
    n_1 = len(bin_1_data)
    n_2 = len(bin_2_data)
    v_1 = np.mean([(x - m_1) ** 2 for x in bin_1_data])
    v_2 = np.mean([(x - m_2) ** 2 for x in bin_2_data])
    v_p = ((n_1 - 1)*v_1 + (n_2 - 1)*v_2)/(n_1 + n_2 - 2)
    Z = (m_1 - m_2) / np.sqrt(v_p * (1 / n_1 + 1 / n_2))
    p_value = 1 - norm.cdf(Z)
    return p_value

p_value_table = []
for i in range(1, len(stack)):
    if stack[i-1][0] != stack[i-1][1] and stack[i][0] != stack[i][1]:
        p_value = p_value_cal(stack[i-1], stack[i], Class)
    else:
        p_value = 2
        
    if stack[i-1][1] - stack[i-1][0] + 1 < min_bin_size or stack[i-1][2] < ind_var_rate or stack[i][1] - stack[i][0] + 1 < min_bin_size or stack[i][2] < ind_var_rate:
        p_value = p_value + 1
        
    p_value_table.append([p_value,stack[i-1]])
    
    if len(p_values_table) > 0:
        

SyntaxError: incomplete input (540307863.py, line 27)